# 2. Тематическое моделирование

In [1]:
import json

import bz2
import regex
from tqdm import tqdm
from scipy import sparse

In [2]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
responses = []
with bz2.BZ2File('banki_responses.json.bz2', 'r') as thefile:
    for row in tqdm(thefile):
        resp = json.loads(row)
        if not resp['rating_not_checked'] and (len(resp['text'].split()) > 0):
            responses.append(resp)

201030it [03:06, 1076.94it/s]


### 2.1. Постройте несколько тематических моделей коллекции документов с разным числом тем

In [77]:
### предобработка ###
texts = [] #слепливаем один список все тексты
for i in responses:
    texts.append(i.get('text'))
len(texts)

153499

In [78]:
grade_5 = [] #фильтруем отзывы с оценкой 1
for i in responses:
    if i.get('rating_grade') == 5:
        grade_5.append(i.get('text'))

In [79]:
len(grade_5)

14713

In [80]:
grade_5_clean = []
for i in grade_5:
    grade_5_clean.append(re.sub(r'[^a-zA-Zа-яА-ЯеЁ ]', '', i))
grade_5_clean[0:2]

['Досрочно закрывали вклады перед отпуском Жена заказывала заранее но забыла заказать на меня Сотрудник банка СПБ Ленсовета  Наталья не отказала мне хотя сумма значительная Работает быстро профессионально Ничего лишнего Жаль что прикрыли прием вкладов в офисе Озерки  доступность офиса для меня Заметил что и несут деньги во вклады и берут кредиты Желаю успехов банку',
 'Хотел открыть вклад Доходный под  год  проценты начисляются лесенкой  месяца  потом  месяца  потом  месяца  Но есть условие Общий объем всех дополнительных взносов не может превышать  от суммы первоначального взноса по вкладу А мне нужен вклад с возможностью пополнения Хорошо значит сразу надо делать большой первоначальный взнос пришел в офис банка в Нижнем Новгороде спросил у операциониста Могу ли я открыть вклад Доходный и внести на него деньги межбанковским переводом ответ Да можете Мне открыли  счет вклада  текущий счет  счет карты саму карту не выдали А теперь пройдите в кассу чтобы внести наличными сумму вклада Как

In [81]:
grade_5_clean_lower = []
for i in grade_5_clean:
    grade_5_clean_lower.append(i.lower())
grade_5_clean_lower[0]

'досрочно закрывали вклады перед отпуском жена заказывала заранее но забыла заказать на меня сотрудник банка спб ленсовета  наталья не отказала мне хотя сумма значительная работает быстро профессионально ничего лишнего жаль что прикрыли прием вкладов в офисе озерки  доступность офиса для меня заметил что и несут деньги во вклады и берут кредиты желаю успехов банку'

In [82]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
grade_5_list = []

for i in grade_5_clean_lower:
    now_list = []
    text = i
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('russian'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    grade_5_list.append(filtered_tokens)
print(grade_5_list[0])

['досрочно', 'закрывали', 'вклады', 'отпуском', 'жена', 'заказывала', 'заранее', 'забыла', 'заказать', 'сотрудник', 'банка', 'спб', 'ленсовета', 'наталья', 'отказала', 'хотя', 'сумма', 'значительная', 'работает', 'быстро', 'профессионально', 'лишнего', 'жаль', 'прикрыли', 'прием', 'вкладов', 'офисе', 'озерки', 'доступность', 'офиса', 'заметил', 'несут', 'деньги', 'вклады', 'берут', 'кредиты', 'желаю', 'успехов', 'банку']


In [83]:
from pymorphy3 import MorphAnalyzer
morph = MorphAnalyzer()

grade_5_lemmas = [] #получаю леммы всего текста
for text in grade_5_list:
    now_list = []
    for token in text:
        now_list.append(morph.normal_forms(token)[0])
    grade_5_lemmas.append(now_list)

print(grade_5_lemmas[0:2])

[['досрочно', 'закрывать', 'вклад', 'отпуск', 'жена', 'заказывать', 'заранее', 'забыть', 'заказать', 'сотрудник', 'банк', 'спб', 'ленсовет', 'наталья', 'отказать', 'хотя', 'сумма', 'значительный', 'работать', 'быстро', 'профессионально', 'лишний', 'жаль', 'прикрыть', 'приём', 'вклад', 'офис', 'озерко', 'доступность', 'офис', 'заметить', 'нести', 'деньга', 'вклад', 'брать', 'кредит', 'желать', 'успех', 'банк'], ['хотеть', 'открыть', 'вклад', 'доходный', 'год', 'процент', 'начисляться', 'лесенка', 'месяц', 'месяц', 'месяц', 'условие', 'общий', 'объём', 'дополнительный', 'взнос', 'превышать', 'сумма', 'первоначальный', 'взнос', 'вклад', 'нужный', 'вклад', 'возможность', 'пополнение', 'значит', 'сразу', 'делать', 'большой', 'первоначальный', 'взнос', 'прийти', 'офис', 'банк', 'нижний', 'новгород', 'спросить', 'операционист', 'мочь', 'открыть', 'вклад', 'доходный', 'внести', 'деньга', 'межбанковский', 'перевод', 'ответ', 'мочь', 'открыть', 'счёт', 'вклад', 'текущий', 'счёт', 'счёт', 'карта'

In [84]:
documents = []
for text in grade_5_lemmas:
    one_document = []
    for word in text:
        one_document.append(word)
    documents.append(' '.join(one_document))
print(documents[0:2])

['досрочно закрывать вклад отпуск жена заказывать заранее забыть заказать сотрудник банк спб ленсовет наталья отказать хотя сумма значительный работать быстро профессионально лишний жаль прикрыть приём вклад офис озерко доступность офис заметить нести деньга вклад брать кредит желать успех банк', 'хотеть открыть вклад доходный год процент начисляться лесенка месяц месяц месяц условие общий объём дополнительный взнос превышать сумма первоначальный взнос вклад нужный вклад возможность пополнение значит сразу делать большой первоначальный взнос прийти офис банк нижний новгород спросить операционист мочь открыть вклад доходный внести деньга межбанковский перевод ответ мочь открыть счёт вклад текущий счёт счёт карта сам карта выдать пройти касса внести наличные сумма вклад касса хотеть сделать межбанк перевод сразу сказать несколько день ставка вкладудоходный снизить почему поставить балл условие вклад объяснить дать подписать договор открыть счетвклад пополнить вклад дать снизить ставка эт

In [97]:
len(documents)

14713

In [89]:
df = pd.DataFrame(documents[0:10000], columns=['text']) #на все памяти не хватает

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Создание объекта TfidfVectorizer
vectorizer = TfidfVectorizer()

# Преобразование текстов в TF-IDF матрицу
tfidf_matrix = vectorizer.fit_transform(df['text'])

# Преобразование матрицы в DataFrame для удобства
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df)

       aa  aalleccc  aas  aav   ab  abalmasik  abatishchev  abel  \
0     0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
1     0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
2     0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
3     0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
4     0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
...   ...       ...  ...  ...  ...        ...          ...   ...   
9995  0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
9996  0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
9997  0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
9998  0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   
9999  0.0       0.0  0.0  0.0  0.0        0.0          0.0   0.0   

      abelyaevotpbankru  abs  ...  ёвр   ёж  ёлка  ёлкин  ёлкипалка  ёлочка  \
0                   0.0  0.0  ...  0.0  0.0   0.0    0.0        0.0     0.0   
1                   0.0  

In [91]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [92]:
dictionary = Dictionary(grade_5_lemmas)
corpus = [dictionary.doc2bow(text) for text in grade_5_lemmas]
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [68]:
from gensim.models import lsimodel

In [93]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=30)

CPU times: total: 24.2 s
Wall time: 23.5 s


In [94]:
lsi.show_topics(5)

[(0,
  '0.168*"карта" + 0.144*"вклад" + 0.134*"кредит" + 0.099*"отделение" + 0.093*"деньга" + 0.089*"офис" + 0.088*"счёт" + 0.086*"очень" + 0.085*"сотрудник" + 0.084*"кредитный"'),
 (1,
  '-0.422*"вклад" + 0.338*"сделка" + 0.303*"кредит" + 0.237*"квартира" + 0.192*"документ" + 0.157*"ипотека" + 0.148*"ипотечный" + 0.126*"заявка" + -0.116*"карта" + -0.113*"открыть"'),
 (2,
  '-0.459*"вклад" + 0.341*"карта" + -0.166*"сделка" + 0.152*"лимит" + 0.146*"банкомат" + 0.125*"кредитный" + -0.107*"открыть" + -0.105*"квартира" + 0.102*"заявление" + 0.102*"ткс"'),
 (3,
  '-0.377*"кредит" + 0.204*"сделка" + -0.175*"вклад" + -0.125*"процент" + -0.120*"сумма" + -0.116*"погашение" + -0.111*"брать" + -0.102*"договор" + 0.099*"благодарность" + -0.098*"погасить"'),
 (4,
  '0.385*"сделка" + 0.279*"вклад" + 0.250*"квартира" + -0.211*"кредит" + 0.166*"карта" + 0.160*"ткс" + 0.134*"ипотека" + 0.127*"лимит" + -0.121*"отделение" + -0.120*"сотрудник"')]

In [95]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=5)

CPU times: total: 12.7 s
Wall time: 9 s


In [96]:
lsi.show_topics(5)

[(0,
  '0.168*"карта" + 0.144*"вклад" + 0.134*"кредит" + 0.099*"отделение" + 0.093*"деньга" + 0.089*"офис" + 0.088*"счёт" + 0.086*"очень" + 0.085*"сотрудник" + 0.084*"кредитный"'),
 (1,
  '0.423*"вклад" + -0.339*"сделка" + -0.303*"кредит" + -0.237*"квартира" + -0.192*"документ" + -0.157*"ипотека" + -0.148*"ипотечный" + -0.125*"заявка" + 0.116*"карта" + 0.113*"открыть"'),
 (2,
  '0.459*"вклад" + -0.340*"карта" + 0.165*"сделка" + -0.152*"лимит" + -0.146*"банкомат" + -0.125*"кредитный" + 0.107*"открыть" + 0.104*"квартира" + -0.103*"заявление" + -0.101*"ткс"'),
 (3,
  '-0.375*"кредит" + 0.206*"сделка" + -0.174*"вклад" + -0.126*"процент" + -0.119*"сумма" + -0.114*"погашение" + -0.110*"брать" + -0.100*"договор" + 0.099*"благодарность" + -0.095*"погасить"'),
 (4,
  '-0.383*"сделка" + -0.281*"вклад" + -0.250*"квартира" + 0.210*"кредит" + -0.166*"карта" + -0.163*"ткс" + -0.138*"ипотека" + -0.126*"лимит" + 0.124*"отделение" + 0.120*"сотрудник"')]

### 2.2. Приведите примеры понятных (интерпретируемых) тем

Интерпретируемые темы: 
1. Ипотека на квартиру
2. Снятие денег с банкомата
3. Взятие кредита и его погашение

### 2.3. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). Можете ли вы их прокомментировать / объяснить?

In [98]:
%%time
lsi = lsimodel.LsiModel(corpus=corpus_tfidf, id2word=dictionary, num_topics=30)

CPU times: total: 9.05 s
Wall time: 4.91 s


In [113]:
print("сбербанк", lsi.show_topics(30)[9], '\n', lsi.show_topics(30)[12], '\n', lsi.show_topics(30)[13], '\n', lsi.show_topics(30)[14])

сбербанк (9, '-0.379*"счёт" + 0.261*"карта" + -0.186*"перевод" + 0.184*"банкомат" + -0.182*"открытие" + -0.169*"документ" + 0.161*"сбербанк" + -0.142*"расчётный" + -0.120*"ткс" + -0.117*"платёж"') 
 (12, '0.271*"сбербанк" + 0.248*"ткс" + -0.209*"сделка" + -0.194*"платёж" + -0.187*"авангард" + -0.166*"лимит" + 0.138*"банкомат" + -0.129*"касса" + -0.125*"заявление" + 0.123*"мы"') 
 (13, '-0.406*"сбербанк" + -0.187*"перевод" + 0.177*"банкомат" + 0.166*"карточка" + -0.127*"очередь" + -0.125*"платёж" + 0.120*"проблема" + -0.118*"касса" + 0.115*"обслуживаться" + -0.114*"сбер"') 
 (14, '0.359*"сбербанк" + 0.287*"вс" + -0.276*"депозит" + 0.252*"ткс" + 0.198*"филиал" + 0.146*"вклад" + -0.101*"офис" + -0.101*"ставка" + 0.100*"ещ" + -0.098*"карта"')


в целом у сбербанка всё вращается примерно вокруг тех же тем, что и у остальных банков, часто шла речь про переводы судя по тому что встречается название нескольких банков наряду со сбером.

In [114]:
print("русский стандарт", lsi.show_topics(30)[26], '\n', lsi.show_topics(30)[28])

русский стандарт (26, '-0.310*"филиал" + -0.307*"перевод" + 0.241*"втб" + 0.232*"карточка" + 0.193*"курс" + 0.173*"платёж" + -0.157*"заявление" + -0.128*"русский" + 0.127*"валюта" + -0.121*"стандарт"') 
 (28, '0.210*"филиал" + -0.183*"связный" + 0.182*"комиссия" + 0.182*"ткс" + 0.165*"платёж" + 0.157*"карточка" + -0.153*"стандарт" + -0.152*"русский" + 0.152*"втб" + -0.151*"кредит"')


 Темы связанные с русским стандартом касаются в основном переводов в другие банки. А ещё можно отметить что несмотря на то что название банка состоит из двух разных слов, они идут в одной теме в обоих случаях.
 
Также могу предположить, что при анализе отзывов с оценкой 1, была бы совсем иная картина, больше про проблемы, а не про ипотеку, кредиты и переводы. Но память на нетбуке против моих дальнейших экспериментов))